# Programming Fundamentals - Project
## GMIT Data Analytics 2020
## Cian Hogan
***

# Project Propostiton

Create sample data of call centre NPS scores.

Dependant variable- Call centre NPS

independant variable- Agent experience/skill, High-Medium-Low

wait times - 0-∞ (centering around a mean value)

issue resolved - Bool Yes/No




## Simulate call centre data
***

We are attempting to generate sample call centre statistics. The four variable that we are going to look at are wait time before the call is answered **waitTime**, experience level of the customer service agent **agentExp**, whether the customers issue was resolved **issueRes** and the net promoter score **NPS**. The first 3 variables are considered independant as the outcome of one does not change the outcome of another. *It could be argued that agent experience could affect the rate at which issues are resolved but in this case we are going to assume they are independant and that issue resolution is based entirely on the customers issue itself.*

The key variable that we will look at is NPS. NPS or net promoter score is dependant on the other factors that we are measuring. If a customer has to wait a long time to speak to an agent, the less experienced the agent they speak with, and if the problem could not be resolved then they are less likely to recommend the service to a friend or familly member.

# NPS net promoter score

In [40]:
import numpy as np
import pandas as pd

We write a function that can generate some sample average customer wait times using the poisson distribution in numpys random package. 

In [41]:
def waitTime(avg, n=1):
    rng = np.random.default_rng()
    
    wait = rng.poisson(lam=avg, size=n)
    
    return wait        

We write a function that can generate whether the call in connected to an agent with high or low experience.

In [42]:
def agentExp(perc, n=1):
    
    rng = np.random.default_rng()
    
    agent = rng.choice(a=["low", "high"], p=perc, size=n)
    
    return agent

We write a function that assigns whether a calls issue is resolved or not based on an average resolution rate.

In [43]:
def issueRes(avgRate, n=1):
    
    rng = np.random.default_rng()
    
    res = rng.choice(a=[True, False], p=[avgRate, 1-avgRate], size=n)
    
    return res

We write a function that uses the 3 functions we wrote earlier to generate a pandas dataframe of 3 variables.

In [49]:
def callData(nCalls, avgWait, agentExp_perc, avgRes):
    
    data = pd.DataFrame()
    
    wait = waitTime(n=nCalls, avg=avgWait)
    
    data["Wait time"] = wait
    
    agent = agentExp(perc=agentExp_perc, n=nCalls)
    
    data["Agent Exp"] = agent
    
    resolve = issueRes(avgRes, nCalls)
    
    data["Issue Resolved"] = resolve
    
    return data

We write a function that takes a 3var dataframe and generates a NPS score based on the variables in the dataframe 

In [47]:
def genNPS(df):
    
    rng = np.random.default_rng()
    
    NPSarr = []

    for item in df.itertuples(index=False):
        waitTime, agent, resolved = item[0], item[1], item[2]
        avgNPS = 10
    
        if not resolved:
            avgNPS -= 3
    
        if waitTime > 90:
            avgNPS -= 1
        
        if agent == "low":
            avgNPS -= 2
        
        NPS = rng.normal(avgNPS)
    
        if NPS > 10:
            NPS = 10
        
        NPSarr.append(int(NPS))
    
    df["NPS"] = NPSarr
    
    return df
    

We write a final function here which builds on the previous functions to build a total 4 var dataframe that encapsulates all the call data we are lloking to create.

In [54]:
def genData(calls, waitTime, agentPerc, avgRes):
    
    df = callData(nCalls=calls, avgWait=waitTime, agentExp_perc=agentPerc, avgRes=avgRes)
    
    data = genNPS(df)
    
    return data

In [55]:
nps_df = genData(100, waitTime = 120, agentPerc= [.50,.50], avgRes=.5)

In [56]:
print(nps_df.head())

   Wait time Agent Exp  Issue Resolved  NPS
0        149      high            True    8
1        107       low            True    6
2        118      high            True    8
3        119       low            True    7
4        119      high            True   10


Reference

HBR https://hbr.org/2003/12/the-one-number-you-need-to-grow

Wait time https://www.callcentrehelper.com/is-there-a-correlation-between-queue-time-and-customer-satisfaction-levels-100270.htm

resolution https://stellaconnect.com/2018/05/27/first-contact-resolution-is-the-king-of-customer-service-metrics-heres-why/

